# Mini Project 1

**2025 Introduction to Quantiative Methods in Finance**

**The Erdös Institute**

**Instructions** Use current stock data to create two potentially profitable investment portfolios. One that is higher risk and one that is lower risk.

-- You are to interpret and explain your interpretation of a high risk profile and low risk profile of a portfolio. You should provide some measurable quantitative data in your explanation.

**Idea**

\\

Let us work with the same set of 5 stocks as in the lecture 2. Let the high risk portfolio involve just maximizing the total return on investments regardless of the volatility and the low risk portfolie involve maximizing the total return while minimizing the volatility! Let us say we invest for one financial year.  

In [ ]:
#Import Packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.optimize import minimize
import datetime as dt
sns.set_style('darkgrid')
import yfinance as yf
import scipy.stats as stats

In [ ]:
tickers = ['AMZN', 'MSFT', 'GOOG', 'HD', 'GM']

#last financial year
start_date = dt.datetime.today() - dt.timedelta(days = 252)
end_date = dt.datetime.today()

stock = yf.download(tickers, start = start_date, end = end_date)

/tmp/ipython-input-19-1547578868.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock = yf.download(tickers, start = start_date, end = end_date)
[*********************100%***********************]  5 of 5 completed


In [ ]:
#calculating return of investment of each stock

In [ ]:
daily_returns = np.log(stock['Close']/stock['Close'].shift(1))
daily_returns = daily_returns.dropna()

annualized_volatility = {ticker: np.std(daily_returns[ticker])*np.sqrt(252) for ticker in tickers}
# Calculate the covariance matrix
covariance_matrix = 252*((daily_returns.cov()))

In [ ]:
# Number of assets
n_assets = len(tickers)

# Define an initial guess for asset weights (e.g., equal weights)
initial_weights = np.array([1/n_assets]*n_assets)

# Define weight constraints
#Sum of weights equals 1
#Do not allocate more than 35% of capital into each index in stock_symbol
constraints1 = ({'type': 'eq', 'fun': lambda weights: np.sum(weights)-1},
               {'type': 'ineq', 'fun': lambda weights: min(weights)-0.1},
               {'type': 'ineq', 'fun': lambda weights: 0.35-max(weights)})

# Define the objective function to minimize portfolio variance
def portfolio_volatility(weights):
    portfolio_std_dev = np.sqrt(np.dot(weights.T,np.dot(covariance_matrix,weights)))
    return portfolio_std_dev

def portfolio_return(weights):
    portfolio_return = -1*np.sum(daily_returns.mean()*weights)*252
    return portfolio_return

#low_risk
result_low_risk = minimize(portfolio_volatility, initial_weights, method='SLSQP', constraints=constraints1)

# Optimal asset weights
optimal_weights = result_low_risk.x
optimal_vol = result_low_risk.fun
print(optimal_weights)
print(optimal_vol)
constraints2 = ({'type': 'eq', 'fun': lambda weights: np.sum(weights)-1},
                {'type': 'ineq', 'fun': lambda weights: min(weights)-0.01},
                {'type': 'ineq', 'fun': lambda weights: 0.99 - max(weights)})
#high_risk
result_high_risk = minimize(portfolio_return, initial_weights, method='SLSQP', constraints=constraints2)
optimal_weights = result_high_risk.x

vol = portfolio_volatility(optimal_weights)
print(optimal_weights)
print(vol)

[0.1        0.10023461 0.11030373 0.35       0.33946166]
0.21707689920302126
[0.01001392 0.01473645 0.01       0.0133399  0.95190973]
0.27198064654156534
